In [9]:

using LinearAlgebra 
using Revise 
using TOML
using DelimitedFiles
using Pkg 
using RigidBodyDynamics
using ForwardDiff
using SparseArrays
using QuadrupedBalance

In [10]:
"""Maximal coorindate lqr backward ricatti""" 
function maximal_coordinate_lqr(Ad, Bd,Cd,D, Q, R, max_iter=10000)
    N = max_iter
    n = size(Q,1)
    m = size(R,1)
    l = size(Cd,2)
    P_prev = zeros(n,n)
    K_prev = zeros(m,n)
    P = zeros(n,n)
    Qn = Q
    β = 1e-5
    K = zeros(m,n)

    P_prev .= Qn
    K_prev .= K 
    for k = (N-1):-1:1
        H = [R+Bd'*P_prev*Bd     Bd'*P_prev*Cd           Bd'*D'; 
             Cd'*P_prev*Bd       β*1.0I(6)+Cd'*P_prev*Cd    Cd'*D'; 
             D*Bd                    D*Cd                        -β*1.0I(6)]

        b = [Bd'*P_prev*Ad; Cd'*P_prev*Ad; D*Ad]

        KLM = H\b
        K = KLM[1:m,:]
        L_ = KLM[m+1:m+l,:]
        M = KLM[m+l+1:end,:]

        P .= Q + K'*R*K + β*L_'*L_ + (Ad-Bd*K-Cd*L_)'*P_prev*(Ad-Bd*K-Cd*L_) - M'*D*(Ad-Bd*K-Cd*L_)
        if norm(K_prev - K) < 1e-8
            println("Backward Ricatti converged in ", k, " iterations")
            return K
        end
        P_prev[:] .= P[:]
        K_prev[:] .= K[:]
    end

    return K
end

maximal_coordinate_lqr

In [11]:
data = TOML.parsefile("ipopt_eq_point.toml") # load eq point 
x_eq = data["x_eq"];
u_eq = data["u_eq"];
λ_eq = data["λ_eq"];

In [12]:
urdfpath = joinpath(@__DIR__, "..", "src","a1","urdf","a1.urdf")
A1mech = parse_urdf(urdfpath, floating=true, remove_fixed_tree_joints=false)
A1 = QuadrupedBalance.UnitreeA1FullBody(A1mech);
foot_contacts = [1, 0, 0, 1] # FR, FL, RR, RL
foot_indices = []
for i in 1:length(foot_contacts)
    if(foot_contacts[i] == 1)
        append!(foot_indices, (i-1)*3 .+ (1:3))
    end 
end 

In [25]:
####### Linearizing about Eq point #########
state_inds = [1:4; 8:19; 20:22; 26:37]
A,B,C = QuadrupedBalance.dynamics_jacobians(A1, x_eq, u_eq, λ_eq, foot_indices)
# D_fd = QuadrupedBalance.dfk_world(x_eq)[foot_indices, :]
D_fd = QuadrupedBalance.spatial_jacobian(A1, x_eq)[foot_indices, state_inds]
attitude_error_jacobian = blockdiag(sparse(QuadrupedBalance.quaternion_differential(x_eq[1:4])), sparse(I(27)) )

31×30 SparseMatrixCSC{Float64, Int64} with 39 stored entries:
⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠢⡀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⡀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⡀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢

In [21]:
A = A[state_inds, state_inds]
B = B[state_inds, :]
C = C[state_inds, :]
D = D_fd * attitude_error_jacobian

####### Discretizing the Dynamics Matrices ####### 
n = 31
m = 12 
n_c = 6 
h = 0.001

O = [A B C]
O = [O; zeros(m+n_c, n+m+n_c)]
O_exp = exp(O.*h)
Ad = O_exp[1:n, 1:n]
Bd = O_exp[1:n, n+1:n+m]
Cd = O_exp[1:n, n+m+1:n+m+n_c];
Ad = attitude_error_jacobian' * Ad * attitude_error_jacobian  # convert quaternion error to 3 param representation
Bd = attitude_error_jacobian' * Bd 
Cd = attitude_error_jacobian' * Cd;

In [22]:
############### Setting LQR Gains ###################
Q_gains = zeros(30); 
Q_gains[1] = (1/deg2rad(2.5)^2) # attitude - x
Q_gains[2] = (1/deg2rad(2.5)^2) # attitude - y
Q_gains[3] = (1/deg2rad(2.5)^2)  # attitude - z 
Q_gains[3 .+ [1,2,3,4]] .= 1 ./    ([deg2rad(2.5), deg2rad(2.5), deg2rad(2.5), deg2rad(2.5)]).^2  # Hips  (FR, FL, RR, RL)
Q_gains[3 .+ [5,6,7,8]] .= 1 ./    ([deg2rad(2.5), deg2rad(2.5), deg2rad(2.5), deg2rad(2.5)]).^2  # Thighs (FR, FL, RR, RL)
Q_gains[3 .+ [9,10,11,12]] .= 1 ./ ([deg2rad(2.5), deg2rad(2.5), deg2rad(2.5), deg2rad(2.5)]).^2   # Calves  (FR, FL, RR< RL)
Q_gains[16] = Q_gains[1] / 1000 # Angular vel - x 
Q_gains[17] = Q_gains[2] / 1000 # Angular vel - y 
Q_gains[18] = Q_gains[3] / 1000 # Angular vel - z
Q_gains[19:end] .= Q_gains[4:15] / 1000 # joint damping

# R_gains = zeros(12);
R_gains[[1,2,3,4]] .= 1 ./    ([8.0, 2.0, 2.0, 8.0]).^2 #  Hips (FR, FL, RR, RL)
R_gains[[5,6,7,8]] .= 1 ./    ([8.0, 2.0, 2.0, 8.0]).^2 #  Thighs
R_gains[[9,10,11,12]] .= 1 ./ ([8.0, 2.0, 2.0, 8.0]).^2 #  Calves

Q = sparse(Diagonal(Q_gains)) 
R = sparse(Diagonal(R_gains)) 
K = maximal_coordinate_lqr(Ad,Bd,Cd,D, Q,R);

Backward Ricatti converged in 9352 iterations


In [23]:
open("maximal_lqr_gain.txt", "w") do io
    writedlm(io, K)
end